In [56]:
import requests
from bs4 import BeautifulSoup
import time

iphone_url = "https://www.ebay.co.uk/sch/i.html?_from=R40&_trksid=p4432023.m570.l1313&_nkw=apple+watch&_sacat=0"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
urls = list()

for i in range(200):
    print(i)
    try:
        search_url = iphone_url + str(i+1)

        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        temp = [a["href"] for a in soup.find_all("a", class_="s-item__link")]

        urls = urls + temp
    except:
        break
    time.sleep(2)

    # Filter out non-product links
urls = [link for link in urls if "ebay.co.uk/itm/" in link]
urls = list(set(urls))
urls


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


['https://www.ebay.co.uk/itm/335090904300?epid=15051667833&hash=item4e04f96cec:g:FUcAAOSwIpJlO4AO&amdata=enc%3AAQAIAAAA4AO07NITlUuAU72DL%2FznSrFXwvit6%2BE46DtXKTxOItMV9bcLu2v4i4n6%2FV5XAkEF745YQ2tRR7O4PjqNRTk0VGDXXeFp3frMcAhDzIh5iPTCmQubd77r5Av67hbezEKrVlPPtM1KJAiQr%2BbqYmnlx63nIYiJV%2FoyLuZ6wcYyZJeoAys4jjsasZOSm%2BqcNqYU6TS%2Fc0vW54Wle5sGxJ0grsdLBtqN5u%2FT2srbJjVJVwQqodBX2DkKRFgG9Yk20rkwxiDiyOa%2FbvFhR%2BxLPSvvAmj13%2BkA9XncLDjvxZqlk%2B6E%7Ctkp%3ABk9SR-aG4YzxYg',
 'https://www.ebay.co.uk/itm/276006718443?epid=6034128802&hash=item4043484beb:g:QPMAAOSwfDFk~bOR&amdata=enc%3AAQAIAAAAwAUawmMbThe4w4zVIChJtO44V93FNiYTpnuzfF2H7paH83pXlJ5y6LNauxDCsmv%2FHWh8iK3mRHeXBjAV8ko66dZJpFSclzfThIaqvh73vRATKAavYx%2F4mTTmocuU7PxY505X1alTSmtueMjMcNz1gsl54XL82i5351%2BLxuzSlgv%2BexhG4cGUtw%2BobfmcIusL56kmOqxx%2FmqhCbYmEww6B51PqhdCxatSVtKcIxPynVBzHv29DfYEVAFATU5De0YMeQ%3D%3D%7Ctkp%3ABk9SR8j35IzxYg',
 'https://www.ebay.co.uk/itm/134571371546?epid=27056909272&hash=item1f55142c1a:g:-p0AAOSwwQdkYBdq&amdata=enc%3A

In [60]:
import pandas as pd

# Initialize an empty list to store the results
#results = []

for url in range(351,len(urls)):
    print(url)
    # Send a GET request to the URL
    response = requests.get(urls[url], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements
    try:
        product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    except:
        continue
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    
    # Get the text from the elements
    try:
        product_title = product_title_element.get_text(strip=True)
    except:
        continue
    price = price_element.find(class_="ux-textspans").text.strip()
    try:
        reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    except:
        continue
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    pages_list = list()
    
    for i in range(pages_count):
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page, headers=headers)
        
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements
        review_elements = soup.find_all(class_="ebay-review-section")
        

        # Loop through the review elements and extract the text
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
                
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            results.append({'Search': 'Apple Watch', 'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})
            df = pd.DataFrame(results)
            df.to_csv('ebay_comments.csv', index=False)
        time.sleep(2)
        
df


348
349
350
351


KeyboardInterrupt: 

In [58]:
#df = pd.DataFrame(results)
df = df.drop_duplicates()
df.to_csv('ebay_comments.csv', index=False)
df

,Search,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,musicmagpie,£634.99,,Probably as I had hoped for as the phone is in...,Great but heavy.,4 stars,stickyprobs,"28 Jun, 2023",Yes
1,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,ssa2234ssa,"Fantastic,new and sealed as described. It come...",Fantastic,5 stars,helwa1078-voxze...,"29 Sep, 2022",Yes
2,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,pre-owned,£634.99,phones2gadgets,It’s a good phone.. Had a new 13 model but fo...,Will do for me for a few years.. hopefully..,5 stars,paulprior249,"20 Jun, 2023",Yes
3,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,phonesdirect_com,£634.99,,Perfect quality as if its new and it has 100% ...,Goood,5 stars,88s275,"16 Aug, 2023",Yes
4,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,sell-phones,I love it fantastic phone,Perfect 👍,5 stars,sarolga,"04 Jun, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...,...
77498,Apple Watch,Apple Watch Series 1 42mm Space Grey Aluminium...,,£15.00,,"Cheaper than series 2, but still takes the lat...",Great item wouldn't be without,5 stars,the999wizard,"29 Jun, 2017",
77499,Apple Watch,Apple Watch Series 1 42mm Space Grey Aluminium...,pre-owned,£15.00,bargainsdirect76,"Delighted with my new Smart watch, can read my...",Apple Watch Series 1,5 stars,stacext_6,"15 Jan, 2020",Yes
77500,Apple Watch,Apple Watch Series 1 42mm Space Grey Aluminium...,,£15.00,,"Great design, could be easier to use, but over...",Great watch,5 stars,westw.jac,"24 Oct, 2017",
77501,Apple Watch,Apple Watch Series 1 42mm Space Grey Aluminium...,pre-owned,£15.00,280671,Received my Apple Watch today and was very ple...,Apple watch,4 stars,imzy110,"16 Aug, 2017",Yes


In [59]:
df.Search.value_counts()

Iphone         51903
AirPods         5231
iPad            2354
Macbook         1331
Apple Watch      497
Name: Search, dtype: int64

In [ ]:
Mac
iPhone
iPad
AirPods
Watch
